<a href="https://colab.research.google.com/github/WilliamMejiaG/Packed_Bed_Membrane_Reactor_Simulation/blob/main/GHSV_PBMR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy import optimize
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [ ]:
%pip install xlsxwriter

     |████████████████████████████████| 153kB 2.7MB/s 


In [ ]:
import xlsxwriter

In [ ]:
def F_CO(VF,D_i,D_o,D_shell,FACT,rho_B,i,j,k,l,m):
    ####################################################################
    #Inlet molar fluxes
    R=8.314472 #J/mol K
    Ar = (np.pi*(D_i**2))/4 #m2
    Dm = (D_o-D_i)/np.log(D_o/D_i)
    Pm = np.pi*Dm
    VSweep = l*VF
    VCO2_0 = VF/(m+1)
    VH2_0 = m*VCO2_0
    nH2_0 = VH2_0/(22400*60) #V in cm3 STP/min
    nCO2_0 = VCO2_0/(22400*60) #V in cm3 STP/min
    nSweep = VSweep/(22400*60) #V in cm3 STP/min
    T = j+273
    
    """
    PACKED BED MEMBRANE REACTOR MODEL
    """
    # ODE definition
    def F(y,X):
        F = np.zeros(11)
        """
        RETENTATE SIDE
        """
        #Molar fluxes retentate
    
        n_H2r = y[0] 
        n_CO2r = y[1] 
        n_MeOHr = y[2]
        n_COr = y[3]
        n_Wr = y[4]
   
        n_Tr = n_H2r+n_CO2r+n_MeOHr+n_COr+n_Wr
    
        TERM = (i/(n_Tr)) 
    
        pH2 = TERM*n_H2r
        pCO2 = TERM*n_CO2r
        pMeOH = TERM*n_MeOHr
        pCO = TERM*n_COr
        pW = TERM*n_Wr

        """
        PERMEATE SIDE
        """
        
        Perm_W = 2.48e-3 #mol/ m2 s bar
        Perm_MeOH =  1.77e-4 #mol/ m2 s bar
        Perm_H2 = 1.38e-5 #mol/ m2 s bar
        Perm_CO2 = 1.38e-5
        #Perm_CO2 = 0
        Perm_CO = 1.38e-5
    
        #Molar fluxes permeate

        n_H2p = y[5] 
        n_CO2p = y[6] 
        n_MeOHp = y[7]
        n_COp = y[8]
        n_Wp = y[9]
        n_swp = y[10]
        n_Tp = n_H2p+n_CO2p+n_MeOHp+n_COp+n_Wp+n_swp
    
        Pp = FACT*i#bar
    
        #Molar fractions Permeate
        y_H2p = n_H2p/n_Tp
        y_CO2p = n_CO2p/n_Tp
        y_MeOHp = n_MeOHp/n_Tp
        y_COp = n_COp/n_Tp
        y_Wp = n_Wp/n_Tp
   
    
        #Partial pressures permeate

        pH2_p = y_H2p*Pp #bar
        pCO2_p = y_CO2p*Pp #bar
        pMeOH_p = y_MeOHp*Pp  #bar
        pCO_p = y_COp*Pp #bar
        pW_p = y_Wp*Pp  #bar
   
        """
        KINETICS
        """
        #Equilibrium constants
        K1 = 10**((3066/T)-10.592)
        K3 = 1/(10**((-2073.00/T)+2.029)) 
        #Parameters
    
        A = 1.07*np.exp(36696.00/(R*T)) #N0 mol/ kg cat s bar^2, Q J/mol
        C = 1.22e10*np.exp(-94765.00/(R*T)) # N0 mol/ kg cat s bar, Q J/mol
        D = 3453.38
        E = 0.499*np.exp(17197.00/(R*T)) # N0 bar^-0.5, Q J/mol
        N = 6.62e-11*np.exp(124119.00/(R*T)) # N0 bar^-1, Q J/mol
    
        #Reaction Equations
        B = 1 + D*(pW/pH2) + E*np.sqrt(pH2) + N*pW
        r1 = (A*pCO2*pH2*(1-((pW*pMeOH)/(K1*(pH2**3)*pCO2))))/(B**3)
        r3 = (C*pCO2*(1-((K3*pW*pCO)/(pH2*pCO2))))/B
    
        """
        Differential equations retentate
        """
        F[0] = -Pm*Perm_H2*(pH2-pH2_p)+Ar*(-3*r1-r3)*(rho_B)
        F[1] = -Pm*Perm_CO2*(pCO2-pCO2_p)+Ar*(-r1-r3)*(rho_B)
        F[2] = -Pm*Perm_MeOH*(pMeOH-pMeOH_p)+Ar*(r1)*(rho_B)
        F[3] = -Pm*Perm_CO*(pCO-pCO_p)+(r3)*Ar*(rho_B)
        F[4] = -Pm*Perm_W*(pW-pW_p)+(r1+r3)*Ar*(rho_B)
    
    
        """
        Differential equations permeate co-current mode
        """
        F[5] = Pm*Perm_H2*(pH2-pH2_p)
        F[6] = Pm*Perm_CO2*(pCO2-pCO2_p)
        F[7] = Pm*Perm_MeOH*(pMeOH-pMeOH_p)
        F[8] = Pm*Perm_CO*(pCO-pCO_p)
        F[9] = Pm*Perm_W*(pW-pW_p)
        F[10] = 0

        return F
    # Initial conditions
    
    y = np.array([nH2_0,nCO2_0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nSweep])
    
    xStart = 0.0
    h = 1.0e-5
    X = np.arange(xStart,k,h,dtype=float)
    Y = odeint(F,y,X)
    
    #FOR FROMENT CO-CURRENT FUNCTION
    n_H2r = Y[:,0]
    n_CO2r = Y[:,1]
    n_MeOHr = Y[:,2]
    n_COr = Y[:,3]
    n_Wr = Y[:,4]
    n_H2p = Y[:,5]
    n_CO2p = Y[:,6]
    n_MeOHp = Y[:,7]
    n_COp = Y[:,8]
    n_Wp = Y[:,9]
    n_swp = Y[:,10]

    n_Tr = n_H2r+n_CO2r+n_MeOHr+n_COr+n_Wr
    n_Tp = n_H2p+n_CO2p+n_MeOHp+n_COp+n_Wp+n_swp
    
    pH2r = (n_H2r/n_Tr)*i
    pCO2r = (n_CO2r/n_Tr)*i
    pMeOHr = (n_MeOHr/n_Tr)*i
    pCOr = (n_COr/n_Tr)*i
    pWr = (n_Wr/n_Tr)*i
    
    pH2p = (n_H2p/n_Tp)*i
    pCO2p = (n_CO2p/n_Tp)*i
    pMeOHp = (n_MeOHp/n_Tp)*i
    pCOp = (n_COp/n_Tp)*i
    pWp = (n_Wp/n_Tp)*i
    pswp = (n_swp/n_Tp)*i
              
    #Graphic for conversion
        
    X_CO2 = ((nCO2_0-(n_CO2r[-1]+n_CO2p[-1]))/nCO2_0)*100      
    S_MeOH = ((n_MeOHr[-1]+n_MeOHp[-1])/(nCO2_0-n_CO2r[-1]))*100
    Y_MeOH = (X_CO2*S_MeOH)/100
    
    
    
    
    FROMENT = "FROMENT"
    CO_CURRENT = "CO_CURRENT"
         
    return X,X_CO2,S_MeOH,Y_MeOH,n_H2r,n_CO2r,n_MeOHr,n_COr,n_Wr,n_H2p,n_CO2p,n_MeOHp,n_COp,n_Wp,n_swp,pH2r,pCO2r,pMeOHr,pCOr,pWr,pH2p,pCO2p,pMeOHp,pCOp,pWp,pswp,CO_CURRENT,FROMENT

In [ ]:
def F_PBR(VF,D_i,D_o,D_shell,FACT,rho_B,i,j,k,l,m):
    ####################################################################
    #Inlet molar fluxes
    R=8.314472 #J/mol K
    Ar = (np.pi*(D_i**2))/4 #m2
    Dm = (D_o-D_i)/np.log(D_o/D_i)
    Pm = np.pi*Dm
    #VSweep = l*VF
    VCO2_0 = VF/(m+1)
    VH2_0 = m*VCO2_0
    nH2_0 = VH2_0/(22400*60) #V in cm3 STP/min
    nCO2_0 = VCO2_0/(22400*60) #V in cm3 STP/min
    #nSweep = VSweep/(22400*60) #V in cm3 STP/min
    T = j+273
    
    """
    PACKED BED MEMBRANE REACTOR MODEL
    """
    # ODE definition
    def F(y,X):
        F = np.zeros(5)
        """
        RETENTATE SIDE
        """
        #Molar fluxes retentate
    
        n_H2r = y[0] 
        n_CO2r = y[1] 
        n_MeOHr = y[2]
        n_COr = y[3]
        n_Wr = y[4]
   
        n_Tr = n_H2r+n_CO2r+n_MeOHr+n_COr+n_Wr
    
        TERM = (i/(n_Tr)) 
    
        pH2 = TERM*n_H2r
        pCO2 = TERM*n_CO2r
        pMeOH = TERM*n_MeOHr
        pCO = TERM*n_COr
        pW = TERM*n_Wr
        
        """
        PERMEATE SIDE
        
        
        Perm_W = 0 #mol/ m2 s bar
        Perm_MeOH =  0 #mol/ m2 s bar
        Perm_H2 = 0 #mol/ m2 s bar
        Perm_CO2 = 0
        Perm_CO = 0
        
        #Molar fluxes permeate

        n_H2p = y[5] 
        n_CO2p = y[6] 
        n_MeOHp = y[7]
        n_COp = y[8]
        n_Wp = y[9]
        n_swp = y[10]
        n_Tp = n_H2p+n_CO2p+n_MeOHp+n_COp+n_Wp+n_swp
    
        Pp = FACT*i#bar
    
        #Molar fractions Permeate
        y_H2p = n_H2p/n_Tp
        y_CO2p = n_CO2p/n_Tp
        y_MeOHp = n_MeOHp/n_Tp
        y_COp = n_COp/n_Tp
        y_Wp = n_Wp/n_Tp
   
    
        #Partial pressures permeate

        pH2_p = y_H2p*Pp #bar
        pCO2_p = y_CO2p*Pp #bar
        pMeOH_p = y_MeOHp*Pp  #bar
        pCO_p = y_COp*Pp #bar
        pW_p = y_Wp*Pp  #bar
        """
        """
        KINETICS
        """
        #Equilibrium constants
        K1 = 10**((3066/T)-10.592)
        K3 = 1/(10**((-2073.00/T)+2.029)) 
        #Parameters
    
        A = 1.07*np.exp(36696.00/(R*T)) #N0 mol/ kg cat s bar^2, Q J/mol
        C = 1.22e10*np.exp(-94765.00/(R*T)) # N0 mol/ kg cat s bar, Q J/mol
        D = 3453.38
        E = 0.499*np.exp(17197.00/(R*T)) # N0 bar^-0.5, Q J/mol
        N = 6.62e-11*np.exp(124119.00/(R*T)) # N0 bar^-1, Q J/mol
    
        #Reaction Equations
        B = 1 + D*(pW/pH2) + E*np.sqrt(pH2) + N*pW
        r1 = (A*pCO2*pH2*(1-((pW*pMeOH)/(K1*(pH2**3)*pCO2))))/(B**3)
        r3 = (C*pCO2*(1-((K3*pW*pCO)/(pH2*pCO2))))/B
    
        """
        Differential equations retentate
        """
        F[0] = Ar*(-3*r1-r3)*(rho_B)
        F[1] = Ar*(-r1-r3)*(rho_B)
        F[2] = Ar*(r1)*(rho_B)
        F[3] = (r3)*Ar*(rho_B)
        F[4] = (r1+r3)*Ar*(rho_B)
    
        """
        
        Differential equations permeate co-current mode
        
        F[5] = Pm*Perm_H2*(pH2-pH2_p)
        F[6] = Pm*Perm_CO2*(pCO2-pCO2_p)
        F[7] = Pm*Perm_MeOH*(pMeOH-pMeOH_p)
        F[8] = Pm*Perm_CO*(pCO-pCO_p)
        F[9] = Pm*Perm_W*(pW-pW_p)
        F[10] = 0
        """
        return F
    # Initial conditions
    
    y = np.array([nH2_0,nCO2_0,0.00,0.00,0.00])
    
    xStart = 0.0
    h = 1.0e-5
    X = np.arange(xStart,k,h,dtype=float)
    Y = odeint(F,y,X)
    
    #FOR FROMENT CO-CURRENT FUNCTION
    n_H2r = Y[:,0]
    n_CO2r = Y[:,1]
    n_MeOHr = Y[:,2]
    n_COr = Y[:,3]
    n_Wr = Y[:,4]
    """
    n_H2p = Y[:,5]
    n_CO2p = Y[:,6]
    n_MeOHp = Y[:,7]
    n_COp = Y[:,8]
    n_Wp = Y[:,9]
    n_swp = Y[:,10]
    """
    n_Tr = n_H2r+n_CO2r+n_MeOHr+n_COr+n_Wr
    
    n_Tp = n_H2p+n_CO2p+n_MeOHp+n_COp+n_Wp+n_swp
    
    pH2r = (n_H2r/n_Tr)*i
    pCO2r = (n_CO2r/n_Tr)*i
    pMeOHr = (n_MeOHr/n_Tr)*i
    pCOr = (n_COr/n_Tr)*i
    pWr = (n_Wr/n_Tr)*i
    
    
    pH2p = (n_H2p/n_Tp)*i
    pCO2p = (n_CO2p/n_Tp)*i
    pMeOHp = (n_MeOHp/n_Tp)*i
    pCOp = (n_COp/n_Tp)*i
    pWp = (n_Wp/n_Tp)*i
    pswp = (n_swp/n_Tp)*i
    
              
    #Graphic for conversion
        
    X_CO2 = ((nCO2_0-n_CO2r[-1])/nCO2_0)*100      
    S_MeOH = ((n_MeOHr[-1])/(nCO2_0-n_CO2r[-1]))*100
    Y_MeOH = (X_CO2*S_MeOH)/100
    
    
    
    
    FROMENT = "FROMENT"
    PACKED_BED_REACTOR = "PACKED_BED_REACTOR"
         
    return X,X_CO2,S_MeOH,Y_MeOH,n_H2r,n_CO2r,n_MeOHr,n_COr,n_Wr,n_H2p,n_CO2p,n_MeOHp,n_COp,n_Wp,n_swp,pH2r,pCO2r,pMeOHr,pCOr,pWr,pH2p,pCO2p,pMeOHp,pCOp,pWp,pswp,PACKED_BED_REACTOR,FROMENT

In [ ]:
def F_PBR1(VF,D_i,D_o,D_shell,FACT,rho_B,i,j,k,l,m):
    ####################################################################
    #Inlet molar fluxes
    R=8.314472 #J/mol K
    Ar = (np.pi*(D_i**2))/4 #m2
    Dm = (D_o-D_i)/np.log(D_o/D_i)
    Pm = np.pi*Dm
    VSweep = l*VF
    VCO2_0 = VF/(m+1)
    VH2_0 = m*VCO2_0
    nH2_0 = VH2_0/(22400*60) #V in cm3 STP/min
    nCO2_0 = VCO2_0/(22400*60) #V in cm3 STP/min
    nSweep = VSweep/(22400*60) #V in cm3 STP/min
    T = j+273
    
    """
    PACKED BED MEMBRANE REACTOR MODEL
    """
    # ODE definition
    def F(y,X):
        F = np.zeros(11)
        """
        RETENTATE SIDE
        """
        #Molar fluxes retentate
    
        n_H2r = y[0] 
        n_CO2r = y[1] 
        n_MeOHr = y[2]
        n_COr = y[3]
        n_Wr = y[4]
   
        n_Tr = n_H2r+n_CO2r+n_MeOHr+n_COr+n_Wr
    
        TERM = (i/(n_Tr)) 
    
        pH2 = TERM*n_H2r
        pCO2 = TERM*n_CO2r
        pMeOH = TERM*n_MeOHr
        pCO = TERM*n_COr
        pW = TERM*n_Wr

        """
        PERMEATE SIDE
        """
        
        Perm_W = 0 #mol/ m2 s bar
        Perm_MeOH =  0 #mol/ m2 s bar
        Perm_H2 = 0 #mol/ m2 s bar
        Perm_CO2 = 0
        Perm_CO = 0
    
        #Molar fluxes permeate

        n_H2p = y[5] 
        n_CO2p = y[6] 
        n_MeOHp = y[7]
        n_COp = y[8]
        n_Wp = y[9]
        n_swp = y[10]
        n_Tp = n_H2p+n_CO2p+n_MeOHp+n_COp+n_Wp+n_swp
    
        Pp = FACT*i#bar
    
        #Molar fractions Permeate
        y_H2p = n_H2p/n_Tp
        y_CO2p = n_CO2p/n_Tp
        y_MeOHp = n_MeOHp/n_Tp
        y_COp = n_COp/n_Tp
        y_Wp = n_Wp/n_Tp
   
    
        #Partial pressures permeate

        pH2_p = y_H2p*Pp #bar
        pCO2_p = y_CO2p*Pp #bar
        pMeOH_p = y_MeOHp*Pp  #bar
        pCO_p = y_COp*Pp #bar
        pW_p = y_Wp*Pp  #bar
   
        """
        KINETICS
        """
        #Equilibrium constants
        K1 = 10**((3066/T)-10.592)
        K3 = 1/(10**((-2073.00/T)+2.029)) 
        #Parameters
    
        A = 1.07*np.exp(36696.00/(R*T)) #N0 mol/ kg cat s bar^2, Q J/mol
        C = 1.22e10*np.exp(-94765.00/(R*T)) # N0 mol/ kg cat s bar, Q J/mol
        D = 3453.38
        E = 0.499*np.exp(17197.00/(R*T)) # N0 bar^-0.5, Q J/mol
        N = 6.62e-11*np.exp(124119.00/(R*T)) # N0 bar^-1, Q J/mol
    
        #Reaction Equations
        B = 1 + D*(pW/pH2) + E*np.sqrt(pH2) + N*pW
        r1 = (A*pCO2*pH2*(1-((pW*pMeOH)/(K1*(pH2**3)*pCO2))))/(B**3)
        r3 = (C*pCO2*(1-((K3*pW*pCO)/(pH2*pCO2))))/B
    
        """
        Differential equations retentate
        """
        F[0] = -Pm*Perm_H2*(pH2-pH2_p)+Ar*(-3*r1-r3)*(rho_B)
        F[1] = -Pm*Perm_CO2*(pCO2-pCO2_p)+Ar*(-r1-r3)*(rho_B)
        F[2] = -Pm*Perm_MeOH*(pMeOH-pMeOH_p)+Ar*(r1)*(rho_B)
        F[3] = -Pm*Perm_CO*(pCO-pCO_p)+(r3)*Ar*(rho_B)
        F[4] = -Pm*Perm_W*(pW-pW_p)+(r1+r3)*Ar*(rho_B)
    
    
        """
        Differential equations permeate co-current mode
        """
        F[5] = Pm*Perm_H2*(pH2-pH2_p)
        F[6] = Pm*Perm_CO2*(pCO2-pCO2_p)
        F[7] = Pm*Perm_MeOH*(pMeOH-pMeOH_p)
        F[8] = Pm*Perm_CO*(pCO-pCO_p)
        F[9] = Pm*Perm_W*(pW-pW_p)
        F[10] = 0

        return F
    # Initial conditions
    
    y = np.array([nH2_0,nCO2_0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nSweep])
    
    xStart = 0.0
    h = 1.0e-5
    X = np.arange(xStart,k,h,dtype=float)
    Y = odeint(F,y,X)
    
    #FOR FROMENT CO-CURRENT FUNCTION
    n_H2r = Y[:,0]
    n_CO2r = Y[:,1]
    n_MeOHr = Y[:,2]
    n_COr = Y[:,3]
    n_Wr = Y[:,4]
    n_H2p = Y[:,5]
    n_CO2p = Y[:,6]
    n_MeOHp = Y[:,7]
    n_COp = Y[:,8]
    n_Wp = Y[:,9]
    n_swp = Y[:,10]

    n_Tr = n_H2r+n_CO2r+n_MeOHr+n_COr+n_Wr
    n_Tp = n_H2p+n_CO2p+n_MeOHp+n_COp+n_Wp+n_swp
    
    pH2r = (n_H2r/n_Tr)*i
    pCO2r = (n_CO2r/n_Tr)*i
    pMeOHr = (n_MeOHr/n_Tr)*i
    pCOr = (n_COr/n_Tr)*i
    pWr = (n_Wr/n_Tr)*i
    
    pH2p = (n_H2p/n_Tp)*i
    pCO2p = (n_CO2p/n_Tp)*i
    pMeOHp = (n_MeOHp/n_Tp)*i
    pCOp = (n_COp/n_Tp)*i
    pWp = (n_Wp/n_Tp)*i
    pswp = (n_swp/n_Tp)*i
              
    #Graphic for conversion
        
    X_CO2 = ((nCO2_0-(n_CO2r[-1]+n_CO2p[-1]))/nCO2_0)*100      
    S_MeOH = ((n_MeOHr[-1]+n_MeOHp[-1])/(nCO2_0-n_CO2r[-1]))*100
    Y_MeOH = (X_CO2*S_MeOH)/100
    
    
    
    
    FROMENT = "FROMENT"
    PACKED_BED_REACTOR = "PACKED_BED_REACTOR"
         
    return X,X_CO2,S_MeOH,Y_MeOH,n_H2r,n_CO2r,n_MeOHr,n_COr,n_Wr,n_H2p,n_CO2p,n_MeOHp,n_COp,n_Wp,n_swp,pH2r,pCO2r,pMeOHr,pCOr,pWr,pH2p,pCO2p,pMeOHp,pCOp,pWp,pswp,PACKED_BED_REACTOR,FROMENT

In [ ]:
def F_CC(VF,D_i,D_o,D_shell,FACT,rho_B,i,j,k,l,m):
    ####################################################################
    #Inlet molar fluxes
    R=8.314472 #J/mol K
    Ar = (np.pi*(D_i**2))/4 #m2
    Dm = (D_o-D_i)/np.log(D_o/D_i)
    Pm = np.pi*Dm
    VSweep = l*VF
    VCO2_0 = VF/(m+1)
    VH2_0 = m*VCO2_0
    nH2_0 = (VH2_0/(22400*60))*(3.6e6) #VH2_0 in cm3 STP/min para dar mmol/h
    nCO2_0 = (VCO2_0/(22400*60))*(3.6e6) #VCO2_0  in cm3 STP/min para dar mmol/h
    nSweep =  (VSweep/(22400*60))*(3.6e6) #VSweep in cm3 STP/min para dar mmol/h
    T = j+273
    
    
    """
    PACKED BED MEMBRANE REACTOR MODEL
    """
    # ODE definition
    def F(y,X):
        F = np.zeros(11)
        """
        RETENTATE SIDE
        """
        Perm_W = (2.48e-3)*(3.6e6)#mmol/ m2 h bar
        Perm_MeOH = (1.77e-4)*(3.6e6) #mmol/ m2 h bar
        Perm_H2 = (1.38e-5)*(3.6e6) #mmol/ m2 h bar
        Perm_CO2 = (1.38e-5)*(3.6e6)
        #Perm_CO2 = (0)*(3.6e6)
        Perm_CO = (1.38e-5)*(3.6e6)
        
        
        #Molar fluxes retentate
    
        n_H2r = y[0] 
        n_CO2r = y[1] 
        n_MeOHr = y[2]
        n_COr = y[3]
        n_Wr = y[4]
   
        n_Tr = n_H2r+n_CO2r+n_MeOHr+n_COr+n_Wr
    
        TERM = (i/(n_Tr)) 
    
        pH2 = TERM*n_H2r
        pCO2 = TERM*n_CO2r
        pMeOH = TERM*n_MeOHr
        pCO = TERM*n_COr
        pW = TERM*n_Wr

        """
        PERMEATE SIDE
        """
        
    
        #Molar fluxes permeate

        n_H2p = y[5] 
        n_CO2p = y[6] 
        n_MeOHp = y[7]
        n_COp = y[8]
        n_Wp = y[9]
        n_swp = y[10]
        n_Tp = n_H2p+n_CO2p+n_MeOHp+n_COp+n_Wp+n_swp
    
        Pp = FACT*i#bar
    
        #Molar fractions Permeate
        y_H2p = n_H2p/n_Tp
        y_CO2p = n_CO2p/n_Tp
        y_MeOHp = n_MeOHp/n_Tp
        y_COp = n_COp/n_Tp
        y_Wp = n_Wp/n_Tp
   
    
        #Partial pressures permeate

        pH2_p = y_H2p*Pp #bar
        pCO2_p = y_CO2p*Pp #bar
        pMeOH_p = y_MeOHp*Pp  #bar
        pCO_p = y_COp*Pp #bar
        pW_p = y_Wp*Pp  #bar
   
        """
        KINETICS
        """
        #Equilibrium constants
        K1 = 10**((3066/T)-10.592)
        K3 = 1/(10**((-2073.00/T)+2.029)) 
        #Parameters
    
        A = 1.07*np.exp(36696.00/(R*T)) #N0 mmol/ kg cat h bar^2, Q J/mol
        C = 1.22e10*np.exp(-94765.00/(R*T)) # N0 mmol/ kg cat h bar, Q J/mol
        D = 3453.38
        E = 0.499*np.exp(17197.00/(R*T)) # N0 bar^-0.5, Q J/mol
        N = 6.62e-11*np.exp(124119.00/(R*T)) # N0 bar^-1, Q J/mol
    
        #Reaction Equations
        B = 1 + D*(pW/pH2) + E*np.sqrt(pH2) + N*pW
        r1 = (3.6e6)*(A*pCO2*pH2*(1-((pW*pMeOH)/(K1*(pH2**3)*pCO2))))/(B**3)
        r3 = (3.6e6)*(C*pCO2*(1-((K3*pW*pCO)/(pH2*pCO2))))/B
    
        """
        Differential equations retentate
        """
        F[0] = -Pm*(Perm_H2)*(pH2-pH2_p)+Ar*(-3*r1-r3)*(rho_B)
        F[1] = -Pm*Perm_CO2*(pCO2-pCO2_p)+Ar*(-r1-r3)*(rho_B)
        F[2] = -Pm*(Perm_MeOH)*(pMeOH-pMeOH_p)+Ar*(r1)*(rho_B)
        F[3] = -Pm*Perm_CO*(pCO-pCO_p)+(r3)*Ar*(rho_B)
        F[4] = -Pm*Perm_W*(pW-pW_p)+(r1+r3)*Ar*(rho_B)
    
    
        """
        Differential equations permeate co-current mode
        """
        F[5] = -Pm*(Perm_H2)*(pH2-pH2_p)
        F[6] = -Pm*Perm_CO2*(pCO2-pCO2_p)
        F[7] = -Pm*(Perm_MeOH)*(pMeOH-pMeOH_p)
        F[8] = -Pm*Perm_CO*(pCO-pCO_p)
        F[9] = -Pm*Perm_W*(pW-pW_p)
        F[10] = 0

        return F
    
    # Initial conditions
    
    xStart = 0.0
    h = 0.0005
    #h = 1.0e-5
    X = np.arange(xStart,k,h,dtype=float)

    def initCond(u): #at z=0
        return np.array([nH2_0,nCO2_0,0.0,0.0,0.0,u[0],u[1],u[2],u[3],u[4],nSweep])

    def r(u): #Boundary conditions residuals
        r = np.zeros(len(u))
        Y = odeint(F,initCond(u),X)
        y = Y[len(Y)-1]
       
        r[0] = y[5]
        r[1] = y[6]
        r[2] = y[7]
        r[3] = y[8]
        r[4] = y[9]
    
        return r

    #Calling N_MeOHp and N_Wp
    a1,b1,c1,d1,n_H2r,n_CO2r,n_MeOHr,n_COr,n_Wr,N_H2p,N_CO2p,N_MeOHp,N_COp,N_Wp,n_swp,pH2r,pCO2r,pMeOHr,pCOr,pWr,pH2p,pCO2p,pMeOHp,pCOp,pWp,pswp,CO_CURRENT,FROMENT = F_CO(VF,D_i,D_o,D_shell,FACT,rho_B,i,j,k,l,m)
    
    N_H2p1 = N_H2p*(3.6e6)
    N_CO2p1 = N_CO2p*(3.6e6)
    N_MeOHp1 = N_MeOHp*(3.6e6)
    N_COp1 = N_COp*(3.6e6)
    N_Wp1 = N_Wp*(3.6e6)
    
    
    # Initial conditions
    u = np.array([N_H2p1[-1],N_CO2p1[-1],N_MeOHp1[-1],N_COp1[-1],N_Wp1[-1]]) #Initial guess
    u = optimize.newton(r,u,tol=1e-06)

    Y = odeint(F,initCond(u),X)
    
    #FOR FROMENT CO-CURRENT FUNCTION
    n_H2r = Y[:,0]
    n_CO2r = Y[:,1]
    n_MeOHr = Y[:,2]
    n_COr = Y[:,3]
    n_Wr = Y[:,4]
    n_H2p = Y[:,5]
    n_CO2p = Y[:,6]
    n_MeOHp = Y[:,7]
    n_COp = Y[:,8]
    n_Wp = Y[:,9]
    n_swp = Y[:,10]
    
    n_Tr = n_H2r+n_CO2r+n_MeOHr+n_COr+n_Wr
    n_Tp = n_H2p+n_CO2p+n_MeOHp+n_COp+n_Wp+n_swp
    
    
    
    pH2r = (n_H2r/n_Tr)*i
    pCO2r = (n_CO2r/n_Tr)*i
    pMeOHr = (n_MeOHr/n_Tr)*i
    pCOr = (n_COr/n_Tr)*i
    pWr = (n_Wr/n_Tr)*i
    
    pH2p = (n_H2p/n_Tp)*i
    pCO2p = (n_CO2p/n_Tp)*i
    pMeOHp = (n_MeOHp/n_Tp)*i
    pCOp = (n_COp/n_Tp)*i
    pWp = (n_Wp/n_Tp)*i
    pswp = (n_swp/n_Tp)*i
              
    #Graphic for conversion
        
    X_CO2 = ((nCO2_0-(n_CO2r[-1]+n_CO2p[0]))/nCO2_0)*100      
    S_MeOH = ((n_MeOHr[-1]+n_MeOHp[0])/(nCO2_0*(X_CO2/100)))*100
    Y_MeOH = (X_CO2*S_MeOH)/100
    
    
    FROMENT = "FROMENT"
    COUNTER_CURRENT = "COUNTER_CURRENT"
         
    return X,X_CO2,S_MeOH,Y_MeOH,n_H2r,n_CO2r,n_MeOHr,n_COr,n_Wr,n_H2p,n_CO2p,n_MeOHp,n_COp,n_Wp,n_swp,pH2r,pCO2r,pMeOHr,pCOr,pWr,pH2p,pCO2p,pMeOHp,pCOp,pWp,pswp,COUNTER_CURRENT,FROMENT

In [ ]:
#GENERAL CENTRAL CONTROL
Pr = np.array([10,30,50])  #bar
a = (250-200)/(100)
#t = np.array([210])
t = np.arange(200,250,a,dtype=float) #C
xStop = np.array([0.10])#m
Volumetric_Flow = np.array([250,500,750,1000]) #cm3 STP min-1
H2_CO2_ratio = np.array([3]) 
Sweep_Feed_ratio = np.array([1]) 


a = []
b = []
c = []

for i in Pr:
  for j in t:
      for k in xStop:
          for VF in Volumetric_Flow:
              for m in H2_CO2_ratio:
                  for l in Sweep_Feed_ratio:
                
                      D_i = 0.009 #m
                      D_o = 0.012 #m
                      D_shell = 0.019#m
                      FACT = 1
                      rho_B= 1142 #kg cat/m3 lecho

                      X,X_CO2,S_MeOH,Y_MeOH,n_H2r,n_CO2r,n_MeOHr,n_COr,n_Wr,n_H2p,n_CO2p,n_MeOHp,n_COp,n_Wp,n_swp,pH2r,pCO2r,pMeOHr,pCOr,pWr,pH2p,pCO2p,pMeOHp,pCOp,pWp,pswp,CO_CURRENT,FROMENT = F_CO(VF,D_i,D_o,D_shell,FACT,rho_B,i,j,k,l,m)
                      X,X_CO22,S_MeOH2,Y_MeOH2,n_H2r2,n_CO2r2,n_MeOHr2,n_COr2,n_Wr2,n_H2p2,n_CO2p2,n_MeOHp2,n_COp2,n_Wp2,n_swp2,pH2r2,pCO2r2,pMeOHr2,pCOr2,pWr2,pH2p2,pCO2p2,pMeOHp2,pCOp2,pWp2,pswp2,COUNTER_CURRENT,FROMENT = F_CC(VF,D_i,D_o,D_shell,FACT,rho_B,i,j,k,l,m)
                      X,X_CO24,S_MeOH4,Y_MeOH4,n_H2r4,n_CO2r4,n_MeOHr4,n_COr4,n_Wr4,n_H2p4,n_CO2p4,n_MeOHp4,n_COp4,n_Wp4,n_swp4,pH2r4,pCO2r4,pMeOHr4,pCOr4,pWr4,pH2p4,pCO2p4,pMeOHp4,pCOp4,pWp4,pswp4,PACKED_BED_REACTOR,FROMENT = F_PBR(VF,D_i,D_o,D_shell,FACT,rho_B,i,j,k,l,m)

                      Results_F_CO = ([k,i,j,VF,m,l,X_CO2,S_MeOH,Y_MeOH,CO_CURRENT,FROMENT])
                      Results_F_CC = ([k,i,j,VF,m,l,X_CO22,S_MeOH2,Y_MeOH2,COUNTER_CURRENT,FROMENT])
                      Results_F_PBR = ([k,i,j,VF,m,l,X_CO24,S_MeOH4,Y_MeOH4,PACKED_BED_REACTOR,FROMENT])

                      a.append(Results_F_CO)
                      b.append(Results_F_CC)
                      c.append(Results_F_PBR)

#PRINTING IN EXCEL CO-CURRENT DATA         
workbook = xlsxwriter.Workbook('CO_TEMP_5.xlsx')   
worksheet1 = workbook.add_worksheet('F_CO') 
row = 0
col = 0

for Reactor_Length, Pressure,Temperature,Volumetric_Flow,H2_CO2_ratio,Sweep_Feed_ratio,Conversion,Selectivity,Yield,CO_CURRENT,FROMENT in a:
    worksheet1.write(row+1,col,Reactor_Length)
    worksheet1.write(row+1,col+1,Pressure)
    worksheet1.write(row+1,col+2,Temperature)
    worksheet1.write(row+1,col+3,Volumetric_Flow)
    worksheet1.write(row+1,col+4,H2_CO2_ratio)
    worksheet1.write(row+1,col+5,Sweep_Feed_ratio)
    worksheet1.write(row+1,col+6,Conversion)
    worksheet1.write(row+1,col+7,Selectivity)
    worksheet1.write(row+1,col+8,Yield)
    worksheet1.write(row+1,col+9,CO_CURRENT)
    worksheet1.write(row+1,col+10,FROMENT)
    row += 1
workbook.close()

#PRINTING IN EXCEL CC-CURRENT DATA  
workbook = xlsxwriter.Workbook('CC_TEMP_5.xlsx')   
worksheet1 = workbook.add_worksheet('F_CC')
row = 0
col = 0

for Reactor_Length, Pressure,Temperature,Volumetric_Flow,H2_CO2_ratio,Sweep_Feed_ratio,Conversion,Selectivity,Yield,COUNTER_CURRENT,FROMENT in b:
    worksheet1.write(row+1,col,Reactor_Length)
    worksheet1.write(row+1,col+1,Pressure)
    worksheet1.write(row+1,col+2,Temperature)
    worksheet1.write(row+1,col+3,Volumetric_Flow)
    worksheet1.write(row+1,col+4,H2_CO2_ratio)
    worksheet1.write(row+1,col+5,Sweep_Feed_ratio)
    worksheet1.write(row+1,col+6,Conversion)
    worksheet1.write(row+1,col+7,Selectivity)
    worksheet1.write(row+1,col+8,Yield)
    worksheet1.write(row+1,col+9,COUNTER_CURRENT)
    worksheet1.write(row+1,col+10,FROMENT)
    row += 1
workbook.close()

#PRINTING IN EXCEL PBR DATA  

workbook = xlsxwriter.Workbook('PBR_TEMP_5.xlsx')   
worksheet1 = workbook.add_worksheet('F_PBR') 

row = 0
col = 0

for Reactor_Length, Pressure,Temperature,Volumetric_Flow,H2_CO2_ratio,Sweep_Feed_ratio,Conversion,Selectivity,Yield,PACKED_BED_REACTOR,FROMENT in c:
    worksheet1.write(row+1,col,Reactor_Length)
    worksheet1.write(row+1,col+1,Pressure)
    worksheet1.write(row+1,col+2,Temperature)
    worksheet1.write(row+1,col+3,Volumetric_Flow)
    worksheet1.write(row+1,col+4,H2_CO2_ratio)
    worksheet1.write(row+1,col+5,Sweep_Feed_ratio)
    worksheet1.write(row+1,col+6,Conversion)
    worksheet1.write(row+1,col+7,Selectivity)
    worksheet1.write(row+1,col+8,Yield)
    worksheet1.write(row+1,col+9,PACKED_BED_REACTOR)
    worksheet1.write(row+1,col+10,FROMENT)
    row += 1
workbook.close()


                    

In [ ]:
from google.colab import files
files.download('CO_TEMP_5.xlsx')
files.download('CC_TEMP_5.xlsx')
files.download('PBR_TEMP_5.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>